In [82]:
import re
from konlpy.tag import Okt
import spacy
import pandas as pd
okt = Okt()

In [73]:
recommend_stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한', '해주다', '과', '영화', '출연', '등장', '하다'])

def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\sSFsf]", "", review)
    word_review = okt.morphs(review_text, stem=True)
    if remove_stopwords:
        word_review = [token for token in word_review if not token in stop_words]
    return word_review

In [81]:
review1 = "유아인이 등장하는 범죄 영화 추천해줘"
review2 = "오달수가 등장하는 코미디 영화 추천해줘"
review3 = "마동석과 유아인이 등장하는 액션 영화 추천해줘"
review4 = "로버트 다우니 주니어가 등장하는 sf SF 영화 추천해줘"
review5 = "다크 판타지 영화 추천해줘"
review6 = "베드 엔딩 영화 추천해줘"
review7 = "마동석이 등장하지 않는 액션 영화 추천해줘"
review8 = "실화 기반 영화 추천해줘"

print(preprocessing(review1, okt, True, recommend_stop_words))
print(preprocessing(review2, okt, True, recommend_stop_words))
print(preprocessing(review3, okt, True, recommend_stop_words))
print(preprocessing(review4, okt, True, recommend_stop_words))
print(preprocessing(review5, okt, True, recommend_stop_words))
print(preprocessing(review6, okt, True, recommend_stop_words))
print(preprocessing(review7, okt, True, recommend_stop_words))
print(preprocessing(review8, okt, True, recommend_stop_words))

['유아인', '범죄', '추천']
['오달수', '코미디', '추천']
['마동석', '유아인', '액션', '추천']
['로버트', '다우니', '주니어', 'sf', 'SF', '추천']
['다크', '판타지', '추천']
['베드', '엔딩', '추천']
['마동석', '않다', '액션', '추천']
['실화', '기반', '추천']


In [69]:
preprocessing(review, okt, True, recommend_stop_words)

['로버트', '다우니', '주니어', 'sf', 'SF', '추천']

In [71]:
preprocessing(review, okt, True, recommend_stop_words)

['다크', '판타지', '추천']

In [74]:
preprocessing(review, okt, True, recommend_stop_words)

['베드', '엔딩', '추천']

In [76]:
preprocessing(review, okt, True, recommend_stop_words)

['마동석', '않다', '액션', '추천']

In [25]:
# 문장에서 키워드 추출
nlp = spacy.load("ko_core_news_sm")
# 장르 동의어 정의
genre_synonyms_ko = {
    "액션": ["액션", "격투", "무협"],
    "코미디": ["코미디", "웃음", "유머"],
    "드라마": ["드라마", "극"],
    "공포": ["공포", "호러", "무서운"],
    "로맨스": ["로맨스", "멜로"],
    "SF": ["SF", "과학", "공상 과학"],
    "스릴러": ["스릴러", "긴장", "서스펜스"],
    "판타지": ["판타지", "환상"],
    "애니메이션": ["애니메이션", "만화"]
}
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한'])

def extract_keywords(sentence):
    doc = nlp(sentence)
    keywords = [token.text.strip() for token in doc if token.text.strip() not in stop_words and token.pos_ in ['NOUN', 'ADJ', 'VERB']]

    # 이름 옆의 불용어 처리
    cleaned_keywords = []
    for keyword in keywords:
        if any(word in keyword for word in stop_words):
            # 공백으로 나누고, 불용어 제거
            parts = re.split(r'\s+', keyword)
            cleaned_parts = [part for part in parts if part not in stop_words]
            cleaned_keyword = ' '.join(cleaned_parts)
            cleaned_keywords.append(cleaned_keyword)
        else:
            cleaned_keywords.append(keyword)

    return cleaned_keywords

In [28]:
# user_input = "유아인이 등장하는 범죄 영화 추천해줘"
# user_input = "오달수가 등장하는 코미디 영화 추천해줘"
# user_input = "마동석과 유아인이 등장하는 액션 영화 추천해줘"
user_input = "마동석이 등장하지 않는 액션 영화 추천해줘"

In [62]:
# 사용자 입력 처리 및 영화 추천
stop_words = {"및", "도", "에", "의", "가", "이", "은", "는", "을", "를", "에서", "로", "과", "와", "한", "그", "이", "하", "여", "속", "전", "자"}
# user_input = input("영화 제목, 원하는 장르, 감독 또는 출연 배우를 입력하세요 (예: 공포, 액션, 멜로, 서부, 로버트 다우니 주니어): ")
user_input = "공포 장르 영화 추천해줘"
user_input = "로버트 다우니 주니어"
# user_input = "백승기 감독의 영화 추천해줘"
# user_input = "유아인 배우가 출연한 영화추천해줘"
# user_input = "유아인이 출연한 범죄 영화 추천해줘" 이건 추가한 거

# 키워드 추출 및 출력
keywords = extract_keywords(user_input)
print(keywords)

로버트 다우니 주니어
['로버트', '다우니']


In [ ]:
import pandas as pd
import spacy
import re

# CSV 파일을 Pandas DataFrame으로 불러오기
file_path = '/content/movie_data'
data = pd.read_csv(file_path)

# 'Unnamed: 0' 열 제거
data = data.drop(columns=['Unnamed: 0'])

# 열 이름을 한국어로 설정
data.columns = ['영화명', '평점', '연도', '상영시간', '연령', '감독', '출연진', '줄거리', '장르']

# spaCy 한국어 모델 로드
nlp = spacy.load("ko_core_news_sm")

# 불용어 정의 (이름이나 중요한 용어는 제외)
stop_words = {"및", "도", "에", "의", "가", "이", "은", "는", "을", "를", "에서", "로", "과", "와", "한", "그", "이", "하", "여", "속", "전", "자"}

# 장르 동의어 정의
genre_synonyms_ko = {
    "액션": ["액션", "격투", "무협"],
    "코미디": ["코미디", "웃음", "유머"],
    "드라마": ["드라마", "극"],
    "공포": ["공포", "호러", "무서운"],
    "로맨스": ["로맨스", "멜로"],
    "SF": ["SF", "과학", "공상 과학"],
    "스릴러": ["스릴러", "긴장", "서스펜스"],
    "판타지": ["판타지", "환상"],
    "애니메이션": ["애니메이션", "만화"]
}

# 장르 정규화 함수
def normalize_genre_from_synonyms_ko(keyword):
    keyword = keyword.strip().lower()
    for genre, synonyms in genre_synonyms_ko.items():
        if keyword in [synonym.lower() for synonym in synonyms]:
            return genre
    return keyword

# 장르 텍스트 정규화
def normalize_genre(text):
    if pd.isna(text):
        return ''
    # 슬래시를 쉼표로 변경하고, 불필요한 공백 제거
    text = re.sub(r'\s*\(/?\s*|\s*/\s*|\s*\)\s*', ',', text)
    text = re.sub(r'\s+', ' ', text)
    genres = [normalize_genre_from_synonyms_ko(genre.strip()) for genre in text.split(',')]
    return ','.join(sorted(set(genres)))

# 문장에서 키워드 추출
def extract_keywords(sentence):
    doc = nlp(sentence)
    keywords = [token.text.strip() for token in doc if token.text.strip() not in stop_words and token.pos_ in ['NOUN', 'ADJ', 'VERB']]

    # 이름 옆의 불용어 처리
    cleaned_keywords = []
    for keyword in keywords:
        if any(word in keyword for word in stop_words):
            # 공백으로 나누고, 불용어 제거
            parts = re.split(r'\s+', keyword)
            cleaned_parts = [part for part in parts if part not in stop_words]
            cleaned_keyword = ' '.join(cleaned_parts)
            cleaned_keywords.append(cleaned_keyword)
        else:
            cleaned_keywords.append(keyword)

    return cleaned_keywords

# 영화 정보 딕셔너리 생성
movie_info = {}
for i, row in data.iterrows():
    movie_id = i
    content = row.to_dict()
    movie_info[movie_id] = content

# 영화 제목과 정보를 매핑하는 새로운 DataFrame 생성
movie_titles = data[['영화명']].rename(columns={'영화명': '제목'})
movie_titles['info'] = movie_titles.index.map(lambda x: movie_info[x])

# DataFrame의 장르 정규화
movie_titles['info'] = movie_titles['info'].apply(lambda x: {**x, '장르': normalize_genre(x['장르'])})

# 장르로 영화 추천
def recommend_movies_by_genre(genre):
    normalized_genre = normalize_genre_from_synonyms_ko(genre)
    recommended_movies = movie_titles[movie_titles['info'].apply(lambda x: '장르' in x and normalized_genre in x['장르'])]
    return recommended_movies[['제목', 'info']]

# 감독 또는 출연 배우로 영화 추천
def recommend_movies_by_person(person_name):
    person_name = person_name.strip()
    recommended_movies = movie_titles[movie_titles['info'].apply(
        lambda x: '감독' in x and person_name in x['감독']
    ) | movie_titles['info'].apply(
        lambda x: '출연진' in x and person_name in x['출연진']
    )]
    return recommended_movies[['제목', 'info']]

# 장르 목록을 가져오는 함수
def get_genres():
    genres = set()
    for info in movie_titles['info']:
        if '장르' in info:
            text = info['장르']
            genres.update([normalize_genre_from_synonyms_ko(genre.strip()) for genre in text.split(',')])
    return sorted(genres)

# 영화 제목으로 정보를 가져오는 함수
def get_movie_info(title):
    filtered_title = title.strip()
    matching_titles = movie_titles[movie_titles['제목'].str.contains(filtered_title, na=False)]

    if matching_titles.empty:
        return f"영화를 찾지 못했습니다: {title}"

    movie_id = matching_titles.index[0]
    return movie_titles[movie_titles.index == movie_id]['info'].iloc[0]

# 사용자 입력 처리 및 영화 추천
user_input = input("영화 제목, 원하는 장르, 감독 또는 출연 배우를 입력하세요 (예: 공포, 액션, 멜로, 서부, 로버트 다우니 주니어): ")

# 키워드 추출 및 출력
keywords = extract_keywords(user_input)

# 감독 및 출연 배우 목록
directors = data['감독'].dropna().apply(lambda x: x.split(', ')).explode().unique()
cast_members = data['출연진'].dropna().apply(lambda x: x.split(', ')).explode().unique()

# 장르와 인물 찾기
genres = get_genres()

genre_found = next((genre for genre in genres if genre in keywords), None)
person_found = next((keyword for keyword in keywords if keyword in directors or keyword in cast_members), None)

if genre_found:
    recommended_movies = recommend_movies_by_genre(genre_found)
    if not recommended_movies.empty:
        result = ""
        for _, row in recommended_movies.iterrows():
            for kind, content in row['info'].items():
                result += f"{kind}: {content}\n"
            result += "-" * 50 + "\n"
    else:
        result = f"추천할 {genre_found} 영화가 없습니다."
elif person_found:
    recommended_movies = recommend_movies_by_person(person_found)
    if not recommended_movies.empty:
        result = ""
        for _, row in recommended_movies.iterrows():
            for kind, content in row['info'].items():
                result += f"{kind}: {content}\n"
            result += "-" * 50 + "\n"
    else:
        result = f"{person_found} 관련 영화를 찾지 못했습니다."
else:
    movie_info = get_movie_info(user_input)
    if "영화를 찾지 못했습니다" not in movie_info:
        result = "영화 정보:\n"
        for kind, content in movie_info.items():
            result += f"{kind}: {content}\n"
    else:
        result = movie_info

print(result)
